In [1]:
PROJECT_ID = "andrewcooley-test-project"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://andrewcooley-reasoning-engine"

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [2]:
from googleapiclient import discovery
from IPython.display import display, Markdown
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain_core import prompts
from langchain.memory import ChatMessageHistory
from vertexai.preview import reasoning_engines

In [3]:
model = "gemini-1.5-flash-001"

In [4]:
DATA_STORE_ID = "kaggle-movies_1717439038857"
LOCATION_ID = "global" 

In [13]:
def search_kaggle_movies(query: str) -> str:
    """
    Search across a movie database given a natural language text query. The query can search for many different attributes about a movie.

    Args:
        query (str): the user's query text to find results in the movie database.

    Returns:
        str: the search results for the user's query.
    """
    from langchain_google_community import VertexAISearchRetriever

    retriever = VertexAISearchRetriever(
        project_id=PROJECT_ID,
        data_store_id=DATA_STORE_ID,
        location_id=LOCATION_ID,
        engine_data_type=1,
        max_documents=10,
    )

    result = str(retriever.invoke(query))
    return result

In [14]:
search_kaggle_movies("basketball")

'[Document(page_content=\'{"id": "14736", "poster_path": "https://image.tmdb.org/t/p/original/zNZWNX19FZ5QyedprVM0ldsXFiP.jpg", "runtime": "124", "tagline": "All\\\'s fair in love and basketball.", "budget": "20000000", "original_language": "en", "status": "Released", "popularity": 5.733227, "original_title": "Love & Basketball", "vote_count": "93", "spoken_languages": "[{\\\'iso_639_1\\\': \\\'en\\\', \\\'name\\\': \\\'English\\\'}, {\\\'iso_639_1\\\': \\\'es\\\', \\\'name\\\': \\\'Espa\\\\u00f1ol\\\'}]", "imdb_id": "tt0199725", "release_date": "2000-04-21", "production_countries": "[{\\\'iso_3166_1\\\': \\\'US\\\', \\\'name\\\': \\\'United States of America\\\'}]", "title": "Love & Basketball", "revenue": "27728118", "overview": "A young African-American couple navigates the tricky paths of romance and athletics in this drama. Quincy McCall (Omar Epps) and Monica Wright (Sanaa Lathan) grew up in the same neighborhood and have known each other since childhood. As they grow into adulth

In [52]:
# Define prompt template
prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (
        lambda x: format_to_openai_function_messages(x["intermediate_steps"])
    ),
} | prompts.ChatPromptTemplate.from_messages(
    [
        ("system", "You are a search and recommendation assistant for a movie database. You must explain, with a summary, why you recommend specific titles to a user. You must also use a user's previous behavior to influence recommendations."),
        ("placeholder", "{history}"),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
        
    ]
)

# Initialize session history
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [49]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0.5},
    tools=[search_kaggle_movies],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [55]:
response = agent.query(
    input="Okay. I like dinosaur movies.",
    config={"configurable": {"session_id": "demo3456"}},
)

# display(Markdown(response["output"]))
response

Parent run b39183bf-7a31-4752-95b7-7c1e758b70ef not found for run a5064c41-6329-43cc-b7f3-0bb506a13dca. Treating as a root run.


{'input': 'Okay. I like dinosaur movies.',
 'history': [{'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'kwargs': {'content': 'I need a movie suggestion.', 'type': 'human'}},
  {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'kwargs': {'content': 'What kind of movie are you in the mood for?  I need some more information to give you a good recommendation. For example, what genres do you like? Are you looking for something lighthearted or something more serious? \n',
    'type': 'ai',
    'tool_calls': [],
    'invalid_tool_calls': []}},
  {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'kwargs': {'content': "Can't you use my past queries to do that?",
    'type': 'human'}},
  {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'kwargs': {'content': 'I am sorry, I do not have access to you

In [56]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0.5},
    tools=[search_kaggle_movies],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [60]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform==1.51.0",
        "langchain==0.1.20",
        "langchain-google-vertexai==1.0.3",
        "cloudpickle==3.0.0",
        "pydantic==2.7.1",
        "langchain_google_community",
        "google-cloud-discoveryengine",
    ],
)

Using bucket andrewcooley-reasoning-engine
Writing to gs://andrewcooley-reasoning-engine/reasoning_engine/reasoning_engine.pkl
Writing to gs://andrewcooley-reasoning-engine/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://andrewcooley-reasoning-engine/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/619758184732/locations/us-central1/reasoningEngines/6483248322948628480/operations/744402666590830592
ReasoningEngine created. Resource name: projects/619758184732/locations/us-central1/reasoningEngines/6483248322948628480
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/619758184732/locations/us-central1/reasoningEngines/6483248322948628480')


In [58]:
# Retrieve the project number associated with your project ID
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=PROJECT_ID)
response = request.execute()
project_number = response["projectNumber"]
project_number

'619758184732'

In [59]:
# Add a new role binding to the IAM policy
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role=roles/discoveryengine.editor

Updated IAM policy for project [andrewcooley-test-project].
bindings:
- members:
  - serviceAccount:service-619758184732@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-619758184732@gcp-sa-vertex-ex-cc.iam.gserviceaccount.com
  role: roles/aiplatform.extensionCustomCodeServiceAgent
- members:
  - serviceAccount:service-619758184732@gcp-sa-vertex-ex.iam.gserviceaccount.com
  role: roles/aiplatform.extensionServiceAgent
- members:
  - serviceAccount:service-619758184732@gcp-sa-vertex-rag.iam.gserviceaccount.com
  role: roles/aiplatform.ragServiceAgent
- members:
  - serviceAccount:service-619758184732@gcp-sa-vertex-eval.iam.gserviceaccount.com
  role: roles/aiplatform.rapidevalServiceAgent
- members:
  - serviceAccount:service-619758184732@gcp-sa-aiplatform-re.iam.gserviceaccount.com
  role: roles/aiplatform.reasoningEngineServiceAgent
- members:
  - serviceAccount:service-619758184732@gcp-sa-aiplatform.ia

In [78]:
response = remote_agent.query(
    input="Cool! Can you recommend any movies from the 1990s about time travel?",
    config={"configurable": {"session_id": "demo_1"}},
)

display(Markdown(response["output"]))

Based on your interest in time travel movies, I recommend checking out "Back to the Future Part II" (1989). It's a fun and exciting sequel to the original "Back to the Future" that sees Marty McFly and Doc Brown travel to 2015.  It's a classic time travel movie with a lot of humor and heart.  Would you like to know more? 


In [87]:
def extract_content(data):
    """Extracts content strings from the history in the provided data structure.

    Args:
        data: A dictionary containing conversation history.

    Returns:
        A list of strings containing the message content from the history.
    """
    contents = []
    for message in data['history']:
        if 'kwargs' in message and 'content' in message['kwargs']:
            contents.append(message['kwargs']['content'])
    return contents

# Example usage with your data (assuming it's stored in a variable named 'data')
contents = extract_content(response)
for content in contents:
    print(content)

How about documentaries about nature?
I found a few documentaries about nature that might interest you, like "Planet Earth" and "Planet Earth II". These are both highly acclaimed series that showcase the beauty and diversity of our planet. "Animals Are Beautiful People" is a classic nature documentary about wildlife in Southern Africa, and "Wild China" is a six-part series on the natural history of China.  If you're looking for something more recent, "Born in China" is a heartwarming documentary that follows a panda bear and her cub, a golden monkey, and a snow leopard.  Let me know if you'd like to explore any of these further! 

Are any of those narrarated by David Attenborough?
You're right!  "Planet Earth" and "Planet Earth II" are both narrated by David Attenborough.  He's a legend in the world of nature documentaries.  Would you like to learn more about those? 

